### imports

In [1]:
import pandas as pd
import numpy as np
import sys
import os

sys.path.insert(0, os.path.abspath(''))
from python_scripts.utilities.api_calls import get_cookie_from_blob, fetch_items, fetch_item_to_df


### running it back

In [2]:
def simulate_portfolio(items, initial_balance, start_date, end_date):
    start_date, end_date = pd.to_datetime(start_date), pd.to_datetime(end_date)
    print(f"\nSimulation: {start_date.date()} to {end_date.date()}")
    print(f"Initial Balance: ${initial_balance:,.2f}\n")

    cookie = get_cookie_from_blob()
    allocation = initial_balance / len(items)
    final_value = uninvested = initial_balance

    for item in items:
        data = fetch_item_to_df(item, cookie)
        if start_date in data.index and end_date in data.index:
            buy, sell = data.loc[start_date, 'price_usd'], data.loc[end_date, 'price_usd']
            if np.isfinite(buy) and np.isfinite(sell) and buy > 0:
                quantity = int(allocation // buy)
                invested = quantity * buy
                uninvested -= invested
                value = quantity * sell
                final_value += value - invested
                print(f"{item}: Bought {quantity} @ ${buy:.2f}, Sold @ ${sell:.2f}, Net: ${value-invested:,.2f}")
            else:
                print(f"{item}: Invalid price data (Likely NaN)")
        else:
            print(f"{item}: Insufficient data")

    total_return = (final_value - initial_balance) / initial_balance
    annual_return = (1 + total_return) ** (365 / (end_date - start_date).days) - 1

    print(f"\nFinal Value: ${final_value:,.2f}")
    print(f"Total Return: {total_return:.2%}")
    print(f"Annualized Return: {annual_return:.2%}")


### input (usage)

In [3]:
# Usage
items = ["Glove Case Key", "Officer Jacques Beltram | Gendarmerie Nationale", "Kilowatt Case", "AK-47 | Blue Laminate (Factory New)", "Glove Case",
         "★ StatTrak™ Paracord Knife | Case Hardened (Field-Tested)"]

simulate_portfolio(items, 10000, '2023-01-01', '2024-01-01')



Simulation: 2023-01-01 to 2024-01-01
Initial Balance: $10,000.00

Glove Case Key: Bought 209 @ $7.97, Sold @ $16.75, Net: $1,835.23
Officer Jacques Beltram | Gendarmerie Nationale: Bought 649 @ $2.56, Sold @ $6.81, Net: $2,756.30
Kilowatt Case: Insufficient data
AK-47 | Blue Laminate (Factory New): Bought 251 @ $6.64, Sold @ $9.73, Net: $775.59
Glove Case: Bought 688 @ $2.42, Sold @ $3.67, Net: $859.31
★ StatTrak™ Paracord Knife | Case Hardened (Field-Tested): Invalid price data (Likely NaN)

Final Value: $16,226.43
Total Return: 62.26%
Annualized Return: 62.26%
